# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
#!pip install ipywidgets

In [2]:
# imports
import transformers
import pandas as pd
from transformers import pipeline
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
from transformers import DistilBertForSequenceClassification

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [3]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng.insert(0, 'Original Language',['en','en','en','en','en','en','en','en','en','en'], True)


    #TODO 1: Add your code here
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", header = 0, names = ['Movie / TV Series', 'Year', 'Synopsis', 'Review'])
    df_fr.insert(0, 'Original Language',['fr','fr','fr','fr','fr','fr','fr','fr','fr','fr'], True)
    df_sp = pd.read_csv("data/movie_reviews_sp.csv", header = 0, names = ['Movie / TV Series', 'Year', 'Synopsis', 'Review'])
    df_sp.insert(0, 'Original Language',['sp','sp','sp','sp','sp','sp','sp','sp','sp','sp'], True)
    df = pd.concat([df_eng, df_fr, df_sp],ignore_index=True)
    return df

df = preprocess_data()

In [4]:
df.sample(5)

,Original Language,Movie / TV Series,Year,Synopsis,Review
17,fr,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues..."
22,sp,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued..."
13,fr,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous..."
20,sp,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu..."
3,en,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ..."


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [5]:
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded


/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df['Review'][df['Original Language']=='fr']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df['Synopsis'][df['Original Language']=='fr']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))


# filter reviews in Spanish and translate to English
es_reviews = df['Review'][df['Original Language']=='sp']
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df['Synopsis'][df['Original Language']=='sp']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x,es_en_model, es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
reviews = pd.concat([df.Review[:10], fr_reviews_en, es_reviews_en],ignore_index=True)
synopses = pd.concat([df.Synopsis[:10], fr_synopsis_en, es_synopsis_en],ignore_index=True)

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
df.update(reviews.to_frame())
df.update(synopses.to_frame())
df.sample(5)

,Original Language,Movie / TV Series,Year,Synopsis,Review
8,en,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
21,sp,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ..."
14,fr,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ..."
29,sp,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ..."
5,en,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina..."


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [8]:
# TODO 5: Update the code below
# load sentiment analysis model
sentiment_classifier = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")
# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    return classifier(text)[0]['label']



In [9]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
sentiments = df.Review.apply(lambda x: analyze_sentiment(x, sentiment_classifier))
df.insert(len(df.columns), 'Sentiment', sentiments.tolist(), True)

In [10]:
df.sample(5)

,Original Language,Movie / TV Series,Year,Synopsis,Review,Sentiment
25,sp,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",NEGATIVE
5,en,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",NEGATIVE
2,en,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",POSITIVE
4,en,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",POSITIVE
13,fr,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",POSITIVE


In [11]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)